# Radar Detection Modeling

## Introduction

In this notebook, we cover a set of standard models for radar detection in the presence of noise.  These include the Swerling radar fluctuation models [[1]](#swerling_rm-1217) and the Marcum (non-fluctuating) model [[2]](#marcum_rm-753).

In each case, we focus on finding a balance between setting a detection threshold above the mean system noise (regardless of what that mean is) that provides an acceptably low probability of a false noise detection with an acceptable probability of a successful target detection.  For many applications, probabilities of false alarm tend to be on the order of $10^{-8}$ to $10^{-3}$ and probabilities of target detection tend to be above $0.5$.

Here, we cover the case where the fundamental mean system noise is Gaussian and is estimated perfectly.  A follow-on effort will cover the case where we are limited to a finite number of radar detection cells to estimate the mean of the total system noise (external and internal).

We also focus here on the approach described in Mitchell and Walker [[3]](#mitchell-walker), which addresses the case where we have perfect knowledge of system noise and extends that to cases where system noise can only be estimated given a finite number of radar returns well away from any target.

## Probability of False Alarm

The probability of false alarm given radar receiver noise and a square law detector is given by:

$P_{fa} = \frac{1}{(N-1)!} \int_{t}^{\infty} V^{N-1} e^{-V} dV $

where $N$ is the number of non-coherently integrated pulses and $t$ is a pre-determined detection threshold (Ref 1 with change of notation per Ref 2).  Note that this is a version of upper [incomplete gamma function](https://en.wikipedia.org/wiki/Incomplete_gamma_function "Incomplete Gamma Function") that has been normalized by $ \frac{1}{(N-1)!} $.  Note also that the [gamma_inc](https://juliamath.github.io/SpecialFunctions.jl/stable/functions_list/#SpecialFunctions.gamma_inc "Julia gamma_inc Function") function in Julia returns a tuple of both the upper and lower incomplete gamma function _ratios_, defined to be the respective incomplete gamma functions normalized by the Gamma function, $ \Gamma(a) $. We note that $ \Gamma(n) = (n-1)! $ for integer values of $ n > 0 $.  Thus, the probability of false alarm in the equation above is directly calculated by [gamma_inc](https://juliamath.github.io/SpecialFunctions.jl/stable/functions_list/#SpecialFunctions.gamma_inc "Julia gamma_inc Function") with the number of non-coherently integrated pulses, $ N $, as the first parameter and the pre-determined detection threshold, $ t $, as the second.  Note also that we only need the upper incomplete gamma function, which is the second value in the return tuple.

In [1]:
function Pfa(t,N)
  return gamma_inc(N,t,0)[2]
end

Pfa (generic function with 1 method)

## Detection Threshold

To calculate the required detection threshold to achieve a specified probability of false alarm, we just need to solve the integral equation above for $ t $.  Fortunately Julia has the [gamma_inc_inv](https://juliamath.github.io/SpecialFunctions.jl/stable/functions_list/#SpecialFunctions.gamma_inc_inv "Inverse Incomplete Gamma Function") function that provides this.  The first parameter is the number of non-coherently integrated pulses ( $ N $ ), the second parameter is $ 1 - P_{fa} $, and the third parameter is $ P_{fa} $, where $ P_{fa} $ is the desired probability of false alarm.

In [2]:
function dt(pfa,N)
    return gamma_inc_inv(N,1-pfa,pfa);
end

dt (generic function with 1 method)

## Swerling Model Implementation
There are numerous approaches for solving the Swerling model equations numerically.  Here we will implement a general purpose approach following Mitchell and Walker [[3]](#mitchell-walker).  We start with the case where the estimated mean of the receiver noise is identical to the actual mean of the receiver noise.  A future effort may also capture the case in the documentation where the receiver noise is estimated separately by a finite length cell-averaging Constant False Alarm Rate (CFAR) process.  Note that this initial case is equivalent to an infinite length cell-averaging CFAR.

## Chi-square Signal Plus Noise Distribution for Radar

Per Mitchell and Walker [[3]](#mitchell-walker), the general probability of detection of any radar signal in the presence of gaussian receiver noise is

$ P_D(\bar{X},N) = \sum_{j=0}^\infty a(j,\bar{X})g(N+j,t) $

where $\bar{X}$ is the average signal to noise ratio over the calculation interval, $N$ is the number of non-coherently integrated pulses during that calculation interval,

$ g(n,t) = e^{-t}\sum_{s=0}^{n-1} \frac{t^s}{s!} $

and

$ a(j,\bar{X}) = \frac{1}{j!} \int_{0}^{\infty} X^j e^{-X}w(X,\bar{X})dX $

$ w(X,\bar{X}) = \frac{1}{\Gamma(K)} \left(K/\bar{X}\right)^K X^{K-1} e^{-KX/\bar{X}} $ for $X>0$

where $K$ is a target fluctuation parameter.  [[3]](#mitchell-walker) also provides recursive formulas for computing these which we will look at next.

Cases of interest include:

<table style="width:40%" align="left">
<tr>
<td style="text-align:left">$0<K<1$</td>
<td style="text-align:left">Weinstock case</td>
</tr>
<tr>
<td style="text-align:left">$K=1$</td>
<td style="text-align:left">Swerling case 1</td>
</tr>
<tr>
<td style="text-align:left">$K=2$</td>
<td style="text-align:left">Swerling case 3</td>
</tr>
<tr>
<td style="text-align:left">$K=N$</td>
<td style="text-align:left">Swerling case 2</td>
</tr>
<tr>
<td style="text-align:left">$K=2N$</td>
<td style="text-align:left">Swerling case 4</td>
</tr>
<tr>
<td style="text-align:left">$K \rightarrow \infty$</td>
<td style="text-align:left">Marcum (or non-fluctuating) case</td>
</tr>
</table>

### Recursive formulas[[3]](mitchell-walker)

Reference [[3]](#mitchell-walker) identifies the following recursion formula for $g(n,t)$ and $a(j,\bar{X})$.

$g(n+1,t) = g(n,t) + t^ne^{-t}/n!$ for $n \geq 1$ and $g(1,t) = e^{-t}$

$a(j+1,\bar{X},K) = \frac{\bar{X}(1+j/K)}{\left(1+\bar{X}/K\right)(1+j)}a(j,\bar{X},K)$ for $j \geq 0$

and

$a(0,\bar{X},K) = \left(1+\bar{X}/K\right)^{-K}$ for finite $K$

$a(0,\bar{X},K) = e^{-\bar{X}}$ for $K \rightarrow \infty$

### Probability of Detection (finite $K$)

We cover the case of finite $K$ first.  Note that the formulas in [[3]](mitchell-walker) and repeated above include the $\bar{X}$ term, which is the average signal to noise ratio after integration.  It is more common in radar analysis to use the average single pulse signal to noise ratio as per the code below.

In [3]:
function Pd(x, t, N, K)
    
    # calculate average integrated signal to noise ratio
    x_bar = x*N
    
    # set calculation accuracy
    err = 1e-14
    
    # initialize g(n,t) for the n=0 case
    e = exp(-t)
    g = e;
    
    # define and initialize a new function to support the interation of g(n,t)
    h = e;
    
    # iterate to determine g(N,t)
    for i = 1:N-1
        h *= t/i
        g += h
    end
    
    # initialize a(j,X_bar) for the j=0 case and initialize the result of the Pd calculation
    a = (1+x_bar/K)^(-K)
    result = a*g
    l = 1
    j = 0
    
    # iterate until the last term is less than the set error
    while l > err     
        h *= t/(N+j)
        g += h
        a *= x_bar*(1+j/K) / (1+x_bar/K) / (1+j)
        l = a*g
        result += l
        j += 1
    end
    return result
end

Pd (generic function with 1 method)

### Required Signal to Noise Ratio

Here we need to do essentially the opposite of the $P_d$ calculation.

Given that the $P_d$ calculations are well behaved, it is tempting to implement this via a Newton-Raphson method, where we start with an initial guess $x_0$ for a zero of the function of interest and refine it iteratively as such

$ x_{n+1} = x_n - \frac{f\left(x_n\right)}{f'\left(x_n\right)} $.

In our case, $f(x) = P_d(x) - p$ and $p$ is the desired probability of detection. Note that parameters irrelevant to the iteration are not shown.

It's tempting to use _CalculusWithJulia_ to estimate the $f'(x)$ term, but this doesn't work well out of the box as the calculation doesn't converge within a reasonable amount of time, or even an unreasonable amount of time.  It's not clear to me yet whether it is the $f'(x)$ estimation that isn't converging (quickly), if it's the Newton-Raphson method itself, or both.  Not sure when/if I'll get around to figuring this out.

One approach to address this is to adapt the $P_d$ calculation above to return both the probability of detection and its derivative (or their ratio) simultaneously.  That also remains work to be done.

Until then, a bit more of a brute force approach is offered here.  We note that the $P_d$ calculation above is monotonically increasing with increasing SNR for any choice of other relevant parameters.  We start then with a reasonable initial guess and find solution bounds above and below the valid result based on this initial guess.  We then iteratively update the guess by selecting an updated SNR guess between these bounds, calculating the resulting $P_d$, and setting this as the new upper or lower bound depending on the results.  The iteration ends when the difference between the upper and lower bounds is less than the specified error term.

In [4]:
function SNR(pd, t, N, K)
    err = 1e-14
    guess = t/N
    result = Pd(guess, t, N, K)
    # brute force approach to quickly(?) find upper and lower bounds around the resulting solution
    scale = 2
    lower_bound = guess
    lower_result = result
    while lower_result > pd
        lower_bound /= scale
        lower_result = Pd(lower_bound, t, N, K)
        scale+=1
    end
    scale = 2
    upper_bound = guess
    upper_result = result
    while upper_result < pd
        upper_bound *= scale
        upper_result = Pd(upper_bound, t, N, K)
        scale+=1
    end
    # iterate to find the solution
    while(upper_bound-lower_bound>err)
        guess = 0.5*(upper_bound+lower_bound)
        result = Pd(guess, t, N, K)
        if(result > pd)
            upper_bound = guess
        else
            lower_bound = guess
        end
    end
    return 0.5*(upper_bound+lower_bound)
end

SNR (generic function with 1 method)

## Examples

In [5]:
# some stuff up front
dB(x) = 10. * log10(x)
adB(x) = 10. ^ ( x / 10. )
using SpecialFunctions

### Probability of False Alarm and Detection Threshold

In [6]:
pfa = 1e-6
N = 10
t = dt(pfa,N)
(t, dB(t))

(32.71034051752392, 15.146850651922968)

### Probability of Detection and Required Signal to Noise Ratio

Mitchell and Walker [[3]](#mitchell-walker) includes the following probability of detection values given perfect system noise estimation (infinite CFAR cells) and Swerling I, Swerling II, and Non-flucuating cases.  Those results are tabulated below.

<table align="left">
    <thead>
        <tr>
            <th colspan="5"style="text-align:left">Swerling I</th>
        </tr>
        <tr>
            <th colspan="4" style="text-align:left">Required SNR (dB)</th>
        </tr>
        <tr>
            <th>N</th>
            <th>Pd=0.10</th>
            <th>Pd=0.50</th>
            <th>Pd=0.90</th>
            <th>Pd=0.99</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1</td>
            <td>7.0</td>
            <td>12.8</td>
            <td>21.1</td>
            <td>31.2</td>
        </tr>
        <tr>
            <td>3</td>
            <td>3.3</td>
            <td>8.9</td>
            <td>17.3</td>
            <td>27.0</td>
        </tr>
        <tr>
            <td>10</td>
            <td>-0.4</td>
            <td>5.1</td>
            <td>13.7</td>
            <td>23.8</td>
        </tr>
        <tr>
            <td>30</td>
            <td>-3.5</td>
            <td>2.2</td>
            <td>10.3</td>
            <td>20.6</td>
        </tr>
        <tr>
            <td>100</td>
            <td>-6.6</td>
            <td>-1.0</td>
            <td>6.9</td>
            <td>17.5</td>
        </tr>
    </tbody>
</table>

<table align="left">
    <thead>
        <tr>
            <th colspan="5"style="text-align:left">Swerling II</th>
        </tr>
        <tr>
            <th colspan="4" style="text-align:left">Required SNR (dB)</th>
        </tr>
        <tr>
            <th>N</th>
            <th>Pd=0.10</th>
            <th>Pd=0.50</th>
            <th>Pd=0.90</th>
            <th>Pd=0.99</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1</td>
            <td>8.7</td>
            <td>11.2</td>
            <td>13.2</td>
            <td>14.5</td>
        </tr>
        <tr>
            <td>3</td>
            <td>5.0</td>
            <td>7.4</td>
            <td>9.2</td>
            <td>10.3</td>
        </tr>
        <tr>
            <td>10</td>
            <td>1.6</td>
            <td>3.8</td>
            <td>5.3</td>
            <td>6.4</td>
        </tr>
        <tr>
            <td>30</td>
            <td>-1.3</td>
            <td>0.6</td>
            <td>2.0</td>
            <td>3.0</td>
        </tr>
        <tr>
            <td>100</td>
            <td>-4.3</td>
            <td>-2.6</td>
            <td>-1.2</td>
            <td>-0.3</td>
        </tr>
    </tbody>
</table>

<table align="left">
    <thead>
        <tr>
            <th colspan="5"style="text-align:left">Non-fluctuating Case (not yet implemented)</th>
        </tr>
        <tr>
            <th colspan="4" style="text-align:left">Required SNR (dB)</th>
        </tr>
        <tr>
            <th>N</th>
            <th>Pd=0.10</th>
            <th>Pd=0.50</th>
            <th>Pd=0.90</th>
            <th>Pd=0.99</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1</td>
            <td>8.7</td>
            <td>11.2</td>
            <td>13.2</td>
            <td>14.5</td>
        </tr>
        <tr>
            <td>3</td>
            <td>5.0</td>
            <td>7.4</td>
            <td>9.2</td>
            <td>10.3</td>
        </tr>
        <tr>
            <td>10</td>
            <td>1.6</td>
            <td>3.8</td>
            <td>5.3</td>
            <td>6.4</td>
        </tr>
        <tr>
            <td>30</td>
            <td>-1.3</td>
            <td>0.6</td>
            <td>2.0</td>
            <td>3.0</td>
        </tr>
        <tr>
            <td>100</td>
            <td>-4.3</td>
            <td>-2.6</td>
            <td>-1.2</td>
            <td>-0.3</td>
        </tr>
    </tbody>
</table>

In [7]:
pd = 0.5
pfa = 1e-6
N = 10
t = dt(pfa,N)

K = 1 # Swerling I
#K = N # Swerling II
#K = 2 # Swerling III
#K = 2*N # Swerling IV

snr = SNR(pd, t, N, K)
pd = Pd(snr,t,N,K)
(snr, dB(snr), pd)

(3.301208879734931, 5.186730046230271, 0.4999999999999998)

## References
<a id="references"></a>

<a id="swerling_rm-1217">[1]</a>
Swerling, P., "Probability of Detection for Fluctuating Targets," U.S. Air Force Project RAND Research Memorandum RM-1217, 17 March 1954.

<a id="marcum_rm-753">[2]</a>
Marcum, J. I., "A Statistical Theory of Target Detection by Pulsed Radar: Mathematical Appendix," U.S. Air Force PROJECT RAND Research Memorandum RM-753, 1 July 1948, Equation 39, p. 11.

<a id="mitchell-walker">[3]</a>
Mitchell, R. L. and J. F. Walker, "Recursive Methods for Computing Detection Probabilities." IEEE Transactions on Aerospace and Electronic Systems, Vol AES-7, No. 4, July 1971, pp. 671-676, Equation 10, p. 672.